# The analysis of the density modulation example

This notebook shows various analyses of the results of the multiscale model with customised density profile. First, we investigate the pulse shaping due to the non-linear propagation, the plasma channel. Then we analyse the build-up of the XUV signal and its structure. FInally, we will get more insight directly into the generating process inside the medium by studying the spectra.

## Load libraries & initial data

In [ ]:
## python modules used within this notebook
import numpy as np
from scipy import integrate
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib.animation
import matplotlib.colors as colors
import os
import h5py
import sys
import MMA_administration as MMA
import mynumerics as mn
import units
from IPython.display import display, Markdown
from IPython.display import HTML

import dataformat_CUPRAD as dfC
import HHG
import plot_presets as pp 

anims_author = 'Jan Vábek'

matplotlib.rcParams['animation.embed_limit'] = 200.
%matplotlib inline

Here we dicide if we `show` animations here or `save` them as well.

In [ ]:
visualisation = 'save'

## Load data

We load the data from the pulse propagation in the Pythonic data container. It contains the data about the pulse propagation and some firther characteristics. The data from the micrscopic response and harmonic signal will be loaded later (these data are large, and we will need only a part of them according to the chosen analyses.)

In [ ]:
demos_path = os.path.join(os.environ['MULTISCALE_DEMOS'],'density_profile','15fs')
# demos_path = os.path.join(os.environ['MULTISCALE_WORK_DIR'],'density_profile','15fs')

# h5file1 = os.path.join(demos_path, 'results.h5')
h5file1 = os.path.join(demos_path, 'results_GZIP9.h5')
h5file2 = h5file1
# h5file2 = os.path.join(demos_path, 'results_Hankel.h5')

with h5py.File(h5file1,'r') as f:

    # load cuprad data = pulse propagation
    CUPRAD_res = dfC.get_data(f)
    CUPRAD_res.get_plasma(f)


## Plot the propagating pulse
We choose the time-and-space window to see the pulse as it propagates through the medium. Note that we measure the intensity by the "expected harmonic cutoff", these units are obtained by the formula $E_{\text{cut-off}} = I_P + 3.17U_p$ (it is directly proportional since $U_p$ is linearly proportional to the intensity). Then we plot the plasma channel create by the passage of the pulse. We show both absolute density of free electrons and also relative to the local density.

There are more technical details about the data: We plot the pulse directly as it is stored in the file. This means that we a co-moving frame defined by the group velocity, $v_g$, of the pulse: this is the computational window of CUPRAD. The group velocity $v_g$ is defined from the linear dispersion relation and depends on the chosen reference pressure and central wavelength. Possible density modulation is relative to this reference pressure, whcih is the reason we use the average pressure in our examples. Physically speaking, $v_g$ is arbitrary and needs to be considered in further processing. For example, the Pyrhonic class represented by `CUPRAD_res` contains methods to adjust to the reference given by the speed of light (both activelly by changing the data or just by sychronising the clocks in the $t$-grid).

In [ ]:
tlim = np.asarray((-25,25))  # [fs]  the time range for plotting the propagating pulse
rlim = 150                   # [mum] the radial range for plotting the propagating pulse

ani_outpath = os.path.join(os.environ['MULTISCALE_WORK_DIR'],'density_profile', 'export')
if not(os.path.exists(ani_outpath)): os.makedirs(ani_outpath)

## XUV camera

Here we show the far-field XUV spectra together with the build-up of the signal in the generating medium.

These data might be very large. Please specify here the parametes of the plot, so only the necessary subset of data is loaded and processed.

In [ ]:
rmax = 0.007                # [m]    the radial dimesion to read the data
XUV_theta_range = [-4, 4]   # [mrad] the divergence angle for plotting 
orders_to_plot = 4          # the range of the logarithmic plot of the spatially resolved harmonic spectra
# Hmax_plot_linear = 50       # the maximal frequency in the linear plot of the spatially resolved harmonic spectra


kz_step = 10    #       the step in $z$ for plotting (derived from the spacing used in the computational grid)

H_interest = np.asarray([19, 25, 37, 49])   # harmonics for which we show the build-up
multipliers = [1,4,10,150]                  # multipliers applied to the build-up to fit the figure nicely
delta_H = 1.                                # the camera spectral range to analyse signal of the harmoonics of the interest

In [ ]:
# import data & basic analyses
with h5py.File(h5file1,'r') as f1,  h5py.File(h5file2,'r') as f2:
    # load Hankel data = XUV camera
    ogrid_Hankel = f2[MMA.paths['Hankel_outputs']+'/ogrid'][:]
    rgrid_Hankel = f2[MMA.paths['Hankel_outputs']+'/rgrid'][:]
    zgrid_Hankel = f2[MMA.paths['Hankel_outputs']+'/zgrid'][0:-1:kz_step]

    camera_distance = mn.readscalardataset(f1,MMA.paths['Hankel_inputs']+'/distance_FF','N')
    theta_grid_Hankel = np.arctan(rgrid_Hankel/camera_distance)  # recompute the radial grid to the divergence

    Hgrid_Hankel = ogrid_Hankel/CUPRAD_res.omega0

    kr_max = mn.FindInterval(rgrid_Hankel, rmax) + 1
    rgrid_Hankel = rgrid_Hankel[:kr_max]
    theta_grid_Hankel = theta_grid_Hankel[:kr_max]


    cumulative_field =    f2[MMA.paths['Hankel_outputs']+'/cumulative_field'][0:-1:kz_step,:kr_max,:,0] +\
                    1j*f2[MMA.paths['Hankel_outputs']+'/cumulative_field'][0:-1:kz_step,:kr_max,:,1] 

# find maxima of the harmonics of the interest
H_idx = [tuple(mn.FindInterval(Hgrid_Hankel,(H_interest[k1]-delta_H, H_interest[k1]+delta_H))) for k1 in range(len(H_interest))]
H_max_interest = [np.max(np.abs(cumulative_field[:,:,H_idx[k1][0]:H_idx[k1][1]]),axis=(1,2)) for k1 in range(len(H_interest))]    

Combine the plots above (ChatGPT powered):

In [ ]:
# We assume you already have:
# CUPRAD_res, rgrid_Hankel, zgrid_Hankel, cumulative_field, ...
# plus your Hgrid_Hankel, etc.

# We also assume you have Code 2’s helpers like:
# k_t_min, k_t_max, k_r_max, tlim, rlim, ...
# HHG.ComputeCutoff, mn.symmetrize_y, mn.FieldToIntensitySI, ...

# ---------------------------------------------------------
# Merged figure with 3 subplots:
#   ax1 = top-left
#   ax2 = top-right (now replaced by Code 2's pcolormesh)
#   ax3 = bottom row spanning both columns
# ---------------------------------------------------------
# fig = plt.figure(figsize=(14, 6))

k_r_max          = mn.FindInterval(1e6*CUPRAD_res.rgrid ,1.05*rlim)
k_t_min, k_t_max = mn.FindInterval(1e15*CUPRAD_res.tgrid,1.05*tlim)
k_z_plot_CUPRAD  = CUPRAD_res.Nz - 1
k_z_plot_XUV     = len(zgrid_Hankel)-1
Hmax_plot = 40

fig = plt.figure(figsize=(5, 6))

# Define subplots using subplot2grid
# ax1 = plt.subplot2grid((3, 2), (0, 0), rowspan=2)  # Upper left
# ax2 = plt.subplot2grid((3, 2), (0, 1), rowspan=2)  # Upper right  (REPLACED)
# ax3 = plt.subplot2grid((3, 2), (2, 0), colspan=2)  # Bottom, spanning both columns

ax1 = plt.subplot2grid((3, 1), (2, 0))  
ax2 = plt.subplot2grid((3, 1), (0, 0))
ax3 = plt.subplot2grid((3, 1), (1, 0)) 

# -----------------------------
# 1) Top-left subplot (ax1)
#    from Code 1, "spatially resolved XUV spectrum" (linear scale)
# -----------------------------
r_grid_sym, sym_data_1 = mn.symmetrize_y(
    rgrid_Hankel,
    np.abs(cumulative_field[k_z_plot_XUV,:,:]).T
)
# Just for clarity:
#   r_grid_sym is the symmetrical radial axis
#   sym_data_1 is the symmetrical data (XUV spectrum)...

pc1 = ax1.pcolormesh(
    Hgrid_Hankel,
    1e3 * r_grid_sym,
    sym_data_1.T,
    shading='auto'
)

ax1.set_ylim(XUV_theta_range)
ax1.set_xlim(Hgrid_Hankel[0],Hmax_plot)
ax1.set_xlabel('$\omega/\omega_{\mathrm{laser}}$ [-]')
ax1.set_ylabel(r'$\theta$ [mrad]')
ax1.set_title('XUV spectrum')

cbar1 = fig.colorbar(pc1, ax=ax1)
cbar1.ax.set_ylabel(r'$|S_{XUV}|$ [arb.u.]', rotation=90)



# -----------------------------
# 2) Top-right subplot (ax2)
#    REPLACED by Code 2's pcolormesh (radial vs time)
# -----------------------------
# Precompute your symmetrical data for the initial frame=0
data_cutoff_init =1e-18*mn.FieldToIntensitySI(CUPRAD_res.E_zrt[0, :k_r_max, k_t_min:k_t_max])


r_grid_sym_2, sym_data_2 = mn.symmetrize_y(
    1e6*CUPRAD_res.rgrid[:k_r_max],
    data_cutoff_init.T
)

pc2 = ax2.pcolormesh(
    1e15*CUPRAD_res.tgrid[k_t_min:k_t_max],
    r_grid_sym_2,
    sym_data_2.T,
    shading='auto'
)
ax2.set_xlim(tlim)
ax2.set_ylim((-rlim, rlim))
ax2.set_xlabel(r'$t~[\mathrm{fs}]$')
ax2.set_ylabel(r'$\rho~[\mu\mathrm{m}]$')
ax2.set_title("before interaction")

cbar2 = fig.colorbar(pc2, ax=ax2)
cbar2.ax.set_ylabel(r'Intensity [$\mathrm{W/cm^2}$]', rotation=90)

# -----------------------------
# plot 3
# -----------------------------
data_cutoff_init = 1e-18*mn.FieldToIntensitySI(CUPRAD_res.E_zrt[k_z_plot_CUPRAD, :k_r_max, k_t_min:k_t_max])

r_grid_sym_2, sym_data_2 = mn.symmetrize_y(
    1e6*CUPRAD_res.rgrid[:k_r_max],
    data_cutoff_init.T
)

pc3 = ax3.pcolormesh(
    1e15*CUPRAD_res.tgrid[k_t_min:k_t_max],
    r_grid_sym_2,
    sym_data_2.T,
    shading='auto'
)
ax3.set_xlim(tlim)
ax3.set_ylim((-rlim, rlim))
ax3.set_xlabel(r'$t~[\mathrm{fs}]$')
ax3.set_ylabel(r'$\rho~[\mu\mathrm{m}]$')
ax3.set_title("after interaction")

cbar3 = fig.colorbar(pc3, ax=ax3)
cbar3.ax.set_ylabel(r'Intensity [$\mathrm{W/cm^2}$]', rotation=90)

# ax3_tw.legend()

# A suptitle (for entire figure) that we'll update with z as we animate:
# title = fig.suptitle("z={:.2f} mm".format(1e3*CUPRAD_res.zgrid[0]))



fig.tight_layout()
plt.show()



